In [17]:
from pyproj import Transformer
import geopandas as gpd
import concurrent.futures
import numpy as np

def parallelize(f,
                my_iter,
                max_workers=4,
                progressbar=True,
                total=None):
    
    if total is None and progressbar:
        total = len(my_iter)
        
    Pool = concurrent.futures.ThreadPoolExecutor

    with Pool(max_workers=max_workers) as ex:
        if progressbar:
            results = list(tqdm(ex.map(f, my_iter), total=total))
        else:
            results = list(ex.map(f, my_iter))
    return results

In [2]:
import pystac_client
import planetary_computer

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [3]:
locs = gpd.read_file('locations_v1_point.fgb')

In [4]:
locs.shape

(177374, 25)

In [5]:
loc = locs.sample(1, random_state=0).iloc[0]

In [6]:
bounds = loc.xmin, loc.ymin, loc.xmax, loc.ymax
epsg = loc.epsg
tile = loc.tile

In [8]:
transformer = Transformer.from_crs(epsg, 4326, always_xy=True)
ll_bounds = transformer.transform_bounds(*bounds)

ll_bounds

(79.14404945938443, 15.682800408636407, 79.15609270736992, 15.69446741805047)

In [9]:
time_range = "2020-01-01/2021-01-01"

search = catalog.search(collections=["sentinel-2-l2a"],
                        bbox=ll_bounds,
                        datetime=time_range)
items = search.get_all_items()
len(items)

144

In [10]:
items = [it for it in items
         if it.properties['s2:mgrs_tile'] == tile]

In [11]:
len(items)

72

In [12]:
bands = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08',
         'B09', 'B11', 'B12', 'B8A', 'SCL']

In [13]:
assets_10m = ['B02', 'B03', 'B04', 'B08']
assets_20m = ['B05', 'B06', 'B07', 'B09',
              'B11', 'B12', 'B8A', 'SCL']

In [14]:
import rasterio

In [26]:
import rioxarray

In [90]:
import stackstac


# def slice_to_bounds(ds, bounds):
#     x_slice = (ds['x'].values >= bounds[0]) & (ds['x'].values < bounds[2])
#     y_slice = (ds['y'].values >= bounds[1]) & (ds['y'].values < bounds[3])
#     ds = ds.loc[:, :, x_slice, y_slice].astype(np.float32)
#     return ds

# def slice_to_bounds(ds, bounds):
#     x_slice = slice(bounds[0], bounds[2])
#     y_slice = slice(bounds[3], bounds[1])
#     ds = ds.isel({'x': x_slice, 'y': y_slice})
#     return ds
ds = {}
assets = {10: assets_10m,
          20: assets_20m}
xmin, ymin, xmax, ymax = bounds
ds = {}


keep_vars = [
 'time',
 'band',
 'x',
 'y',
 's2:mean_solar_zenith',
 's2:processing_baseline',
 's2:mean_solar_azimuth',
 'eo:cloud_cover'
]

for res in 10, 20:

    ds[res] = stackstac.stack(items, assets=assets[res])
    ds[res] = ds[res].rio.clip_box(xmin, ymin, xmax - res, ymax - res)
    
    for a in ['spec']:
        del ds[res].attrs[a]
    
    for k, v in zip(['bounds', 'epsg', 'tile', 'patch_id'],
                    [bounds, epsg, tile, loc.patch_id]):
        ds[res].attrs[k] = v
        
    ds_vars = list(ds[res].coords.keys())
    drop_vars = [v for v in ds_vars if v not in keep_vars]
    ds[res] = ds[res].drop_vars(drop_vars)
    
    fn = f"{loc.patch_id}_{res}m.nc"
    ds[res].to_netcdf(fn)
    

In [28]:
for res in 10, 20:
    for k, v in zip(['bounds', 'epsg', 'tile', 'patch_id'],
                    [bounds, epsg, tile, loc.patch_id]):
        ds[res].attrs[k] = v

In [29]:
ds[10]

,Array,Chunk
Bytes,18.00 MiB,64.00 kiB
Shape,"(72, 4, 128, 128)","(1, 1, 128, 128)"
Count,6 Graph Layers,288 Chunks
Type,float32,numpy.ndarray


In [31]:
ds = ds[10]

In [32]:
ds

,Array,Chunk
Bytes,18.00 MiB,64.00 kiB
Shape,"(72, 4, 128, 128)","(1, 1, 128, 128)"
Count,6 Graph Layers,288 Chunks
Type,float32,numpy.ndarray


In [40]:
ds = stackstac.stack(items, assets=assets_10m)

In [41]:
x_slice = (ds['x'].values >= bounds[0]) & (ds['x'].values < bounds[2])
y_slice = (ds['y'].values >= bounds[1]) & (ds['y'].values < bounds[3])

wds = ds.loc[:, :, x_slice, y_slice].astype(np.float32)

In [48]:
ds['x'].values

array([199980., 199990., 200000., ..., 309750., 309760., 309770.])

In [47]:
wds['x'].values

array([263990., 264000., 264010., 264020., 264030., 264040., 264050.,
       264060., 264070., 264080., 264090., 264100., 264110., 264120.,
       264130., 264140., 264150., 264160., 264170., 264180., 264190.,
       264200., 264210., 264220., 264230., 264240., 264250., 264260.,
       264270., 264280., 264290., 264300., 264310., 264320., 264330.,
       264340., 264350., 264360., 264370., 264380., 264390., 264400.,
       264410., 264420., 264430., 264440., 264450., 264460., 264470.,
       264480., 264490., 264500., 264510., 264520., 264530., 264540.,
       264550., 264560., 264570., 264580., 264590., 264600., 264610.,
       264620., 264630., 264640., 264650., 264660., 264670., 264680.,
       264690., 264700., 264710., 264720., 264730., 264740., 264750.,
       264760., 264770., 264780., 264790., 264800., 264810., 264820.,
       264830., 264840., 264850., 264860., 264870., 264880., 264890.,
       264900., 264910., 264920., 264930., 264940., 264950., 264960.,
       264970., 2649

In [45]:
x = ds['x'].values

In [46]:
x[x_slice]

array([301100., 301110., 301120., 301130., 301140., 301150., 301160.,
       301170., 301180., 301190., 301200., 301210., 301220., 301230.,
       301240., 301250., 301260., 301270., 301280., 301290., 301300.,
       301310., 301320., 301330., 301340., 301350., 301360., 301370.,
       301380., 301390., 301400., 301410., 301420., 301430., 301440.,
       301450., 301460., 301470., 301480., 301490., 301500., 301510.,
       301520., 301530., 301540., 301550., 301560., 301570., 301580.,
       301590., 301600., 301610., 301620., 301630., 301640., 301650.,
       301660., 301670., 301680., 301690., 301700., 301710., 301720.,
       301730., 301740., 301750., 301760., 301770., 301780., 301790.,
       301800., 301810., 301820., 301830., 301840., 301850., 301860.,
       301870., 301880., 301890., 301900., 301910., 301920., 301930.,
       301940., 301950., 301960., 301970., 301980., 301990., 302000.,
       302010., 302020., 302030., 302040., 302050., 302060., 302070.,
       302080., 3020

In [38]:
bounds

(301100.0, 1734720.0, 302380.0, 1736000.0)

In [ ]:
for res in 10, 20:
    fn = f"{loc.patch_id}_{res}m.nc"
    ds[res].to_netcdf(fn)

In [31]:
import numpy as np

In [32]:
wds = wds.astype(np.float32)

In [33]:
wds

,Array,Chunk
Bytes,18.00 MiB,64.00 kiB
Shape,"(72, 4, 128, 128)","(1, 1, 128, 128)"
Count,6 Graph Layers,288 Chunks
Type,float32,numpy.ndarray


In [63]:
xmin, ymin, xmax, ymax = bounds
wds2 = ds.rio.clip_box(xmin, ymin, xmax - res, ymax - res)
res = 10
wds2 = ds.rio.clip_box(xmin, ymin, xmax - res, ymax - res)

In [64]:
bounds

(301100.0, 1734720.0, 302380.0, 1736000.0)

In [66]:
wds2

,Array,Chunk
Bytes,36.00 MiB,128.00 kiB
Shape,"(72, 4, 128, 128)","(1, 1, 128, 128)"
Count,4 Graph Layers,288 Chunks
Type,float64,numpy.ndarray


In [51]:
wds2['x'].values

array([301100., 301110., 301120., 301130., 301140., 301150., 301160.,
       301170., 301180., 301190., 301200., 301210., 301220., 301230.,
       301240., 301250., 301260., 301270., 301280., 301290., 301300.,
       301310., 301320., 301330., 301340., 301350., 301360., 301370.,
       301380., 301390., 301400., 301410., 301420., 301430., 301440.,
       301450., 301460., 301470., 301480., 301490., 301500., 301510.,
       301520., 301530., 301540., 301550., 301560., 301570., 301580.,
       301590., 301600., 301610., 301620., 301630., 301640., 301650.,
       301660., 301670., 301680., 301690., 301700., 301710., 301720.,
       301730., 301740., 301750., 301760., 301770., 301780., 301790.,
       301800., 301810., 301820., 301830., 301840., 301850., 301860.,
       301870., 301880., 301890., 301900., 301910., 301920., 301930.,
       301940., 301950., 301960., 301970., 301980., 301990., 302000.,
       302010., 302020., 302030., 302040., 302050., 302060., 302070.,
       302080., 3020

In [52]:
bounds

(301100.0, 1734720.0, 302380.0, 1736000.0)

In [54]:
ds['x'].values[-1]

309770.0

In [57]:
ds['y'].values[-1]

1690210.0

In [55]:
ds.spec

RasterSpec(epsg=32644, bounds=(199980.0, 1690200.0, 309780.0, 1800000.0), resolutions_xy=(10.0, 10.0))

In [12]:
from rio_tiler.io import STACReader

In [13]:
df = gpd.GeoDataFrame.from_features(items.to_dict(), crs="epsg:4326")
df

,geometry,datetime,platform,proj:epsg,instruments,s2:mgrs_tile,constellation,s2:granule_id,eo:cloud_cover,s2:datatake_id,...,s2:cloud_shadow_percentage,s2:nodata_pixel_percentage,s2:unclassified_percentage,s2:dark_features_percentage,s2:not_vegetated_percentage,s2:degraded_msi_data_percentage,s2:high_proba_clouds_percentage,s2:reflectance_conversion_factor,s2:medium_proba_clouds_percentage,s2:saturated_defective_pixel_percentage
0,"POLYGON ((79.18337 16.27295, 79.21998 16.26446...",2020-12-29T05:12:19.024000Z,Sentinel-2B,32644,[msi],44PKC,Sentinel 2,S2B_OPER_MSI_L2A_TL_ESRI_20210713T212739_A0199...,2.627451,GS2B_20201229T051219_019922_N03.00,...,0.972553,0.015192,8.990832,0.805392,23.259160,0.0,1.751793,1.033927,0.875319,0.0
1,"POLYGON ((80.15894 15.51551, 80.14738 15.46690...",2020-12-24T05:12:21.024000Z,Sentinel-2A,32644,[msi],44PLC,Sentinel 2,S2A_OPER_MSI_L2A_TL_ESRI_20201224T235249_A0287...,13.048375,GS2A_20201224T051221_028759_N02.12,...,4.990772,0.622695,14.259659,1.818240,13.569649,0.0,9.548941,1.033349,3.493230,0.0
2,"POLYGON ((78.19312 16.26222, 79.21991 16.27335...",2020-12-24T05:12:21.024000Z,Sentinel-2A,32644,[msi],44PKC,Sentinel 2,S2A_OPER_MSI_L2A_TL_ESRI_20201224T234637_A0287...,3.925820,GS2A_20201224T051221_028759_N02.12,...,1.920571,0.000013,4.772533,1.192302,21.298546,0.0,2.698668,1.033349,1.224601,0.0
3,"POLYGON ((80.15913 15.46996, 80.14460 15.40938...",2020-12-19T05:12:19.024000Z,Sentinel-2B,32644,[msi],44PLC,Sentinel 2,S2B_OPER_MSI_L2A_TL_ESRI_20210513T233401_A0197...,55.370486,GS2B_20201219T051219_019779_N02.12,...,0.771709,0.403154,32.711786,0.605481,7.890747,0.0,23.154710,1.032519,21.303743,0.0
4,"POLYGON ((78.19312 16.26222, 79.21991 16.27335...",2020-12-19T05:12:19.024000Z,Sentinel-2B,32644,[msi],44PKC,Sentinel 2,S2B_OPER_MSI_L2A_TL_ESRI_20201220T132224_A0197...,33.136476,GS2B_20201219T051219_019779_N02.12,...,2.840920,0.000003,31.189549,1.238818,12.778743,0.0,21.584471,1.032519,8.873183,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,"POLYGON ((78.19312 16.26222, 79.21991 16.27335...",2020-01-14T05:11:49.024000Z,Sentinel-2B,32644,[msi],44PKC,Sentinel 2,S2B_OPER_MSI_L2A_TL_ESRI_20201002T183533_A0149...,2.556956,GS2B_20200114T051149_014917_N02.12,...,0.129767,0.000000,6.437928,1.452566,38.570306,0.0,0.180630,1.034091,0.223211,0.0
140,"POLYGON ((80.15918 15.45718, 80.14015 15.37704...",2020-01-09T05:12:01.024000Z,Sentinel-2A,32644,[msi],44PLC,Sentinel 2,S2A_OPER_MSI_L2A_TL_ESRI_20201029T161327_A0237...,75.652002,GS2A_20200109T051201_023754_N02.12,...,0.009978,0.345002,6.241716,0.166703,8.171619,0.0,22.495446,1.034305,16.150300,0.0
141,"POLYGON ((78.19312 16.26222, 79.21991 16.27335...",2020-01-09T05:12:01.024000Z,Sentinel-2A,32644,[msi],44PKC,Sentinel 2,S2A_OPER_MSI_L2A_TL_ESRI_20201029T161313_A0237...,13.322517,GS2A_20200109T051201_023754_N02.12,...,0.016121,0.000000,5.427626,1.079758,31.458035,0.0,0.057916,1.034305,0.159850,0.0
142,"POLYGON ((80.15911 15.47326, 80.14302 15.40561...",2020-01-04T05:12:09.024000Z,Sentinel-2B,32644,[msi],44PLC,Sentinel 2,S2B_OPER_MSI_L2A_TL_ESRI_20201002T225212_A0147...,70.955421,GS2B_20200104T051209_014774_N02.12,...,0.476344,0.414076,11.310400,0.237292,12.496378,0.0,62.838799,1.034259,8.037672,0.0


,Array,Chunk
Bytes,0.97 TiB,8.00 MiB
Shape,"(144, 4, 10980, 20982)","(1, 1, 1024, 1024)"
Count,3 Graph Layers,133056 Chunks
Type,float64,numpy.ndarray


ModuleNotFoundError: No module named 'rio_tiler.io'